In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [17]:
df = pd.read_table("train.tsv")
df.head()

,Unnamed: 0,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,0,39.4,4,85,70.0,2070,18.6,78,3,datsun b210 gx
1,3,18.0,6,171,97.0,2984,14.5,75,1,ford pinto
2,9,32.7,6,168,132.0,2910,11.4,80,3,datsun 280-zx
3,11,38.1,4,89,60.0,1968,18.8,80,3,toyota corolla tercel
4,13,28.0,4,98,80.0,2164,15.0,72,1,dodge colt (sw)


In [18]:
y=df["mpg"].values
x=df.drop(["mpg","car name"], axis=1).values

In [19]:
from sklearn.model_selection import train_test_split
# テストデータと学習データ分割
X_train_raw, X_test, y_train_raw, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# early_stopping用の評価データをさらに分割
X_train, X_valid, y_train, y_valid = train_test_split(X_train_raw, y_train_raw, test_size=0.25, random_state=42)

In [20]:
OBJECTIVE_VARIALBLE = "mpg"  # 目的変数
USE_EXPLANATORY = df.columns.drop(["mpg","car name"]) # 説明変数

In [21]:
import lightgbm as lgb
# データをDatasetクラスに格納
dtrain = lgb.Dataset(X_train, label=y_train)  # 学習用
dvalid = lgb.Dataset(X_valid, label=y_valid)  # early_stopping用
# モデル作成
params = {'objective': 'regression',  # 最小化させるべき損失関数
         'random_state': 42,  # 乱数シード
         'boosting_type': 'gbdt',  # boosting_type
         'n_estimators': 10000  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
         }
verbose_eval = 0  # この数字を1にすると学習時のスコア推移がコマンドライン表示される
# early_stoppingを指定してLightGBM学習
lgbr = lgb.LGBMRegressor(**params)
lgbr.fit(X_train, y_train, 
         eval_metric='rmse',  # early_stoppingの評価指標(学習用の'metric'パラメータにも同じ指標が自動入力される)
         eval_set=[(X_valid, y_valid)],
         early_stopping_rounds=10,
         verbose=verbose_eval
         )

LGBMRegressor(n_estimators=10000, objective='regression', random_state=42)

In [22]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# モデル評価
y_pred = lgbr.predict(X_test)

# rmse : 平均二乗誤差の平方根
rmse = mean_squared_error(y_true=y_test, y_pred=y_pred, squared=False)
print('RMSE :',rmse)

#MAE

mae=mean_absolute_error(y_true=y_test, y_pred=y_pred)
print("MAE",mae)

# r2 : 決定係数
r2 = r2_score(y_true=y_test, y_pred=y_pred)
print('R2 :',r2)

RMSE : 3.4284910198699565
MAE 2.421381225057477
R2 : 0.8129784002346979
